# SQL 的五十道練習：初學者友善的資料庫入門

> 子查詢

讀者如果是資料科學的初學者，可以略過下述的程式碼；讀者如果不是資料科學的初學者，欲使用 JupyterLab 執行本章節內容，必須先執行下述程式碼載入所需模組與連接資料庫。

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/twElection2020.db" AS twElection2020;

## 複習一下

在第五章「函數」我們提過複合函數（Composite functions）的概念，意即在函數中包括函數、先後使用多個函數，先使用的函數輸出將會成為後使用的函數輸入。舉例來說，`SUBSTR()` 函數的輸出為 `'Bos'`，成為 `UPPER()` 函數的輸入，最後的輸出為 `'BOS'`。

In [4]:
SELECT 'Boston' AS city,
       UPPER(SUBSTR('Boston', 1, 3)) AS composite_function;

city,composite_function
Boston,BOS


## 子查詢

如果是一段 SQL 敘述中包括另外一段 SQL 敘述、先後使用多個 SQL 敘述，先執行的 SQL 敘述查詢結果將會成回後執行的 SQL 敘述中的依據，這樣的 SQL 敘述結構就稱為子查詢（Subquery）。常見的子查詢結構有三種外型：

1. 接續在 `WHERE` 保留字後的結構外型。

```sql
SELECT columns
  FROM table
 WHERE (SELECT columns FROM table ...);
```

2. 接續在 `SELECT` 保留字後的結構外型。

```sql
SELECT (SELECT columns FROM table ...)
  FROM table;
```

3. 接續在 `FROM` 保留字後的結構外型，這裡要注意的是先前別名是針對欄位名稱，這裡則是將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）的存在。

```sql
SELECT columns
  FROM (SELECT columns FROM table ...) AS alias;
```

## 常見的子查詢應用情境

檢視常見的三種子查詢結構外型，可以概略猜到子查詢的應用情境。接續在 `WHERE` 保留字後的結構外型，應用於篩選資料表觀測值的條件必須要先經過一個 SQL 敘述查詢才能夠建立；接續在 `SELECT` 保留字後的結構外型，應用於衍生計算欄位的算式部分必須要先經過一個 SQL 敘述查詢才能夠獲得；接續在 `FROM` 保留字後的結構外型，應用於將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）對待來取得所需資訊。

情境一是接續在 `WHERE` 保留字後的結構外型，舉例來說，我們想知道片長 `runtime` 最短的電影是哪一部？假定最短片長為 `x`，我們可以寫出以下的 SQL 敘述得到這個問題的答案：

```sql
SELECT title,
       runtime
  FROM movies
 WHERE runtime = x;
```

但是 `x` 必須要先經過一個 SQL 敘述查詢才能得知為多少，獲得關鍵 `x` 的 SQL 敘述為：

In [5]:
SELECT MIN(runtime) AS min_runtime
  FROM movies;

min_runtime
45


接著我們可以將 `x` 替換為 SQL 敘述，並用小括號 `()` 包裝起來，就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [6]:
SELECT title,
       runtime
  FROM movies
 WHERE runtime = (
                     SELECT MIN(runtime) AS min_runtime
                       FROM movies
                 );

title,runtime
Sherlock Jr.,45


值得注意的地方有兩個，一是子查詢的結構外型也只能有一個分號 `;` 來標註 SQL 敘述的結束，因此替換之後要記得只留下最後執行 SQL 敘述的分號。二是替換之後因為排版變得比較亂，這時可以善用 SQLiteStudio 的 Format SQL 功能讓寫作的 SQL 敘述之編排、格式和設計具備更高的可讀性。

![](../images/format-03.png)

![](../images/format-04.png)

情境二是接續在 `SELECT` 保留字後的結構外型，舉例來說，我們想知道在千禧年（西元 2000 年）之後上映的電影佔比為多少？假定在千禧年（西元 2000 年）之後上映的電影有 `x` 部，我們可以寫出以下的 SQL 敘述得到這個問題的答案：

```sql
SELECT x * 1.0 / COUNT(*) AS after_millennium_ratio
  FROM movies;
```

但是 `x` 必須要先經過一個 SQL 敘述查詢才能得知為多少，獲得關鍵 `x` 的 SQL 敘述為：

In [7]:
SELECT COUNT(*) AS count_after_millennium
  FROM movies
 WHERE release_year >= 2000;

count_after_millennium
96


接著我們可以將 `x` 替換為 SQL 敘述，並用小括號 `()` 包裝起來，就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [8]:
SELECT (
           SELECT COUNT( * ) AS count_after_millennium
             FROM movies
            WHERE release_year >= 2000
       )* 1.0 / COUNT( * ) AS after_millennium_ratio
  FROM movies;

after_millennium_ratio
0.384


情境三是接續在 `FROM` 保留字後的結構外型，舉例來說，我們想知道不同年份 `release_year` 上映的電影平均評等有哪些年份是大於等於 8.5 的？在第九章「分組與聚合結果篩選」我們提過針對分組聚合的結果應用 `WHERE` 是不被允許的，應該要改使用分組聚合版本的 `HAVING` 保留字加上帶有聚合函數的條件。

In [9]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year
HAVING AVG(rating) >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


除了前述改使用分組聚合版本的 `HAVING` 保留字加上帶有聚合函數的條件以外，我們也能透過子查詢來完成。假定有一個資料表 `avg_rating_by_release_year` 記錄了不同年份 `release_year` 上映的電影平均評等，我們可以寫出以下的 SQL 敘述得到這個問題的答案：

```sql
SELECT *
  FROM avg_rating_by_release_year
 WHERE avg_rating >= 8.5;
```

但是 `avg_rating_by_release_year` 必須要先經過一個 SQL 敘述查詢才能得知為多少，獲得關鍵 `avg_rating_by_release_year` 的 SQL 敘述為：

```sql
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year;
```

接著我們可以將 `avg_rating_by_release_year` 替換為 SQL 敘述，並用小括號 `()` 包裝起來，並加上別名，就能成功將本來應該分兩次、先後執行的 SQL 敘述，調整為子查詢的結構外型。

In [10]:
SELECT *
  FROM (
           SELECT release_year,
                  AVG(rating) AS avg_rating
             FROM movies
            GROUP BY release_year
       )
       AS avg_rating_by_release_year
 WHERE avg_rating >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


## 重點統整

- 一段 SQL 敘述中包括另外一段 SQL 敘述、先後使用多個 SQL 敘述，先執行的 SQL 敘述查詢結果將會成回後執行的 SQL 敘述中的依據，這樣的 SQL 敘述結構就稱為子查詢（Subquery）。
- 常見的子查詢結構有三種外型：
    1. 接續在 `WHERE` 保留字後的結構外型。
    2. 接續在 `SELECT` 保留字後的結構外型。
    3. 接續在 `FROM` 保留字後的結構外型，這裡要注意的是先前別名是針對欄位名稱，這裡則是將先執行的 SQL 敘述查詢結果視為像資料表（實際上並不是）的存在。
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias,
       CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_n END AS alias
  FROM table
 WHERE conditions
 GROUP BY columns
HAVING conditions
 ORDER BY columns DESC
 LIMIT m;
```
    
## 練習題 30-34

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 30. 從 `nba` 資料庫的 `players` 資料表運用子查詢找出 NBA 中身高最高與最矮的球員是誰，參考下列的預期查詢結果。

預期輸出：(3, 3) 的查詢結果。

firstName,lastName,heightMeters
Isaiah,Thomas,1.75
Kristaps,Porzingis,2.21
Boban,Marjanovic,2.21


### 31. 從 `nba` 資料庫的 `players` 資料表運用子查詢計算球員的國籍佔比，參考下列的預期查詢結果。

預期輸出：(42, 2) 的查詢結果。

country,player_percentage
USA,0.764822134387352
Canada,0.041501976284585
France,0.0177865612648221
Germany,0.0158102766798419
Australia,0.0138339920948617
Serbia,0.00988142292490119
Spain,0.00988142292490119
Nigeria,0.00790513833992095
Turkey,0.00790513833992095
Argentina,0.00592885375494071


### 32. 從 `nba` 資料庫運用子查詢找出 NBA 的場均得分王（`ppg`），參考下列的預期查詢結果。

預期輸出：(1, 2) 的查詢結果。

firstName,lastName
Kevin,Durant


### 33. 從 `nba` 資料庫運用子查詢找出目前布魯克林籃網隊（Brooklyn Nets）的球員名單，參考下列的預期查詢結果。

預期輸出：(16, 2) 的查詢結果。

firstName,lastName
LaMarcus,Aldridge
Kevin,Durant
Goran,Dragic
Blake,Griffin
Patty,Mills
Kyrie,Irving
Andre,Drummond
Seth,Curry
Joe,Harris
Ben,Simmons


### 34. 從 `twElection2020` 資料庫的 `presidential` 資料表計算各組候選人的得票率，參考下列的預期查詢結果。

預期輸出：(3, 2) 的查詢結果。

candidate_id,votes_percentage
1,4.26%
2,38.61%
3,57.13%
